# AlexNet to predict Bounding Boxes
The Architecture of the model is basically alexnet but we removed the FC layers at the end and added layers that are more useful for predicting BB's

In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import tensorflow as tf
from lib.Plot import plot
import matplotlib.pyplot as plt

In [2]:
X_train = np.load("lib/datasets/train_signs3k.npy")
#X_test = np.load("lib/datasets/test_signs.npy")
X_val = np.load("lib/datasets/val_signs3k.npy")
#y_train = np.load("lib/datasets/train_labels_e3k.npy")
#y_test = np.load("lib/datasets/test_labels_e.npy")
#y_val = np.load("lib/datasets/val_labels_e3k.npy")
z_train = np.load("lib/datasets/train_bboxes3k.npy")
#z_test = np.load("lib/datasets/test_bboxes.npy")
z_val = np.load("lib/datasets/val_bboxes3k.npy")

In [3]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, ZeroPadding2D, BatchNormalization, Input, InputLayer

In [4]:
alex_in = Input(shape=X_train.shape[1:])
alex = Conv2D(96,(11,11),strides=4,activation='relu',padding='valid')(alex_in)
alex = MaxPool2D((3,3),strides=2)(alex)
alex = BatchNormalization()(alex)

alex = ZeroPadding2D(padding=2)(alex)
alex = Conv2D(256,(5,5),activation='relu',padding='same')(alex)
alex = MaxPool2D((3,3), strides=2)(alex)
alex = BatchNormalization()(alex)

alex = ZeroPadding2D(padding=1)(alex)
alex = Conv2D(384,(3,3),activation='relu',padding='same')(alex)

alex = ZeroPadding2D(padding=1)(alex)
alex = Conv2D(384,(3,3),activation='relu',padding='same')(alex)

alex = ZeroPadding2D(padding=1)(alex)
alex = Conv2D(256,(3,3),activation='relu',padding='same')(alex)

alex = Flatten()(alex)
alex = Dense(128,activation="relu")(alex)
#alex = Dropout(rate=0.5)(alex)
alex = Dense(64,activation="relu")(alex)
alex = Dense(32,activation="relu")(alex)
# alex = Dropout(rate=0.5)(alex)
alex_out = Dense(4,activation="softmax",name="alex_b")(alex)

sgd = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=0.1, patience=5, verbose=1,
    mode='max', min_delta=0.0001, cooldown=2, min_lr=0)
alex_b = Model(inputs = alex_in, outputs = alex_out)
alex_b.compile(loss='mean_squared_error',optimizer=sgd,metrics=['accuracy'])
epochs = 20
history = alex_b.fit(X_train, z_train, batch_size = 32, epochs= epochs, validation_data=(X_val,z_val),callbacks=[callback])

Epoch 1/20
161/161 [==============================] - 11s 33ms/step - loss: 0.0997 - accuracy: 0.7392 - val_loss: 0.1023 - val_accuracy: 0.7882 - lr: 0.0010
Epoch 2/20
161/161 [==============================] - 4s 24ms/step - loss: 0.0976 - accuracy: 0.7444 - val_loss: 0.0993 - val_accuracy: 0.7882 - lr: 0.0010
Epoch 3/20
161/161 [==============================] - 4s 25ms/step - loss: 0.0975 - accuracy: 0.7574 - val_loss: 0.0988 - val_accuracy: 0.7777 - lr: 0.0010
Epoch 4/20
161/161 [==============================] - 4s 24ms/step - loss: 0.0974 - accuracy: 0.7635 - val_loss: 0.0983 - val_accuracy: 0.7777 - lr: 0.0010
Epoch 5/20
161/161 [==============================] - 4s 24ms/step - loss: 0.0974 - accuracy: 0.7674 - val_loss: 0.0981 - val_accuracy: 0.7847 - lr: 0.0010
Epoch 6/20
159/161 [============================>.] - ETA: 0s - loss: 0.0973 - accuracy: 0.7706
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
161/161 [==============================] - 4s 

In [5]:
alex_b.save("App/lib/models/alex_bb")

INFO:tensorflow:Assets written to: App/lib/models/alex_bb\assets


In [4]:
alex_b = tf.keras.models.load_model('App/lib/models/alex_bb')

In [5]:
X_train = alex_b.predict(X_train)

In [6]:
np.save("lib/datasets/X_train1.npy", X_train)

In [ ]:
bouns = np.load("lib/datasets/train_signs3k.npy")

In [13]:
tmp = [1,2,3,4]
tmp1 = [1,2,3,4]
from sklearn.model_selection import train_test_split
train_name, test_name = train_test_split(tmp, test_size=0.25, random_state=42)
print(train_name,test_name)

[4, 1, 3] [2]
